In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [217]:
from bs4 import BeautifulSoup as bs
import requests
from csv import writer
import pandas as pd
from lxml import html
import numpy as np
import matplotlib.pyplot as plt

In [3]:
url='https://www.bookdepository.com/bestsellers'

In [4]:
resp=requests.get(url)
resp

<Response [200]>

In [7]:
Page = 1
all_Format = []
for Page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={Page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    formats_on_page = soup.select("div.item-info p.format")
    for product_format in formats_on_page:
        all_Format.append(product_format.get_text())
    Page = Page + 1
formats_series = pd.Series(all_Format)
formats_series.value_counts() #Gives count of Formats

Paperback              714
Hardback               295
Board book              31
Game                     2
Cards                    2
Book                     1
Toy                      1
CD-Audio                 1
Novelty book             1
Mixed media product      1
Sheet music              1
dtype: int64

In [8]:
formats_series

0        Hardback
1       Paperback
2       Paperback
3       Paperback
4            Game
          ...    
1045    Paperback
1046     Hardback
1047    Paperback
1048    Paperback
1049    Paperback
Length: 1050, dtype: object

In [9]:
formats_series.value_counts(normalize=True)

Paperback              0.680000
Hardback               0.280952
Board book             0.029524
Game                   0.001905
Cards                  0.001905
Book                   0.000952
Toy                    0.000952
CD-Audio               0.000952
Novelty book           0.000952
Mixed media product    0.000952
Sheet music            0.000952
dtype: float64

In [10]:
Page = 1
all_years = []
for Page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={Page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    years_on_page = soup.select("div.item-info p.published")
    for product_year in years_on_page:
        a = product_year.get_text()
        #print(a)
        py= a.split(" ")[2]
        #print(py)
        all_years.append(py)
        #PY = product_year.get_text().split(" ")[2]
    page = Page + 1
years_series = pd.Series(all_years)
years_series.value_counts()  #Gives count of Years

2022    393
2021     91
2020     79
2018     69
2017     57
2019     51
2016     42
2011     40
2023     37
2015     37
2014     27
2012     21
2013     17
2007     11
2010     10
2003      8
2005      8
2009      8
2008      7
2000      6
2006      6
1995      6
1997      5
2001      3
1998      2
1996      2
1999      2
1987      2
2002      1
2004      1
dtype: int64

In [11]:
years_df = years_series.value_counts().to_frame().reset_index()
years_df.rename(columns={"index":"Year", 0:"Published books"}, inplace=True)
print(years_df)

    Year  Published books
0   2022              393
1   2021               91
2   2020               79
3   2018               69
4   2017               57
5   2019               51
6   2016               42
7   2011               40
8   2023               37
9   2015               37
10  2014               27
11  2012               21
12  2013               17
13  2007               11
14  2010               10
15  2003                8
16  2005                8
17  2009                8
18  2008                7
19  2000                6
20  2006                6
21  1995                6
22  1997                5
23  2001                3
24  1998                2
25  1996                2
26  1999                2
27  1987                2
28  2002                1
29  2004                1


In [15]:
Page = 1
all_titles = []
for Page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={Page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    titles_on_page = soup.select("div.item-info h3.title")
    for product_title in titles_on_page:
        a = product_title.get_text(strip=True)
        #print(a)
        all_titles.append(a)
    Page = Page + 1
title_series = pd.Series(all_titles)
title_series.value_counts()    # Give count of Titles

The Little Book of Common Sense Investing    3
Small Things Like These                      3
Never Finished                               3
Never Split the Difference                   3
The Secret History                           3
                                            ..
Finding the Mother Tree                      1
The Myth of Sisyphus                         1
Minecraft Annual 2023                        1
SPACEBOY                                     1
Jokes for Funny Kids: 8 Year Olds            1
Length: 990, dtype: int64

In [37]:
Page = 1
books_Price = []
for Page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={Page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    a = soup.find("span", class_="sale-price").get_text().split("€")[0].replace(",", ".").lstrip()  #
    #current_price = a.split("€")[0].replace(",", ".")
    print(a)
        
    Page = Page + 1

16.80 
16.80 
10.18 
6.96 
8.72 
6.36 
8.22 
15.57 
11.92 
15.09 
12.51 
50.59 
11.24 
8.81 
12.72 
12.23 
8.57 
35.23 
39.97 
12.82 
56.22 
10.34 
18.76 
7.75 
6.86 
8.35 
3.47 
41.17 
9.66 
7.32 
18.10 
17.06 
9.57 
9.96 
8.12 


In [55]:
# Merging all 4 featues into one program in dictionary and list

page = 1
bestseller_books = []
bestseller_book = {}
for page in range(35):
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    div = soup.find("div", class_="tab-wrap module type-book grid tab--2 tab-active")
    for book in soup.find_all('div', class_="book-item"):  
        bestseller_book["Title"] = book.find("h3", class_="title").get_text(strip=True)
        #print(bestseller_book["title"])
        bestseller_book["Format"] = book.find("p", class_="format").get_text()
        #print(bestseller_book["format"])
        try:
            bestseller_book["Year"] = book.find("p", class_="published").get_text()[-4:]
        except AttributeError:
            bestseller_book["Year"] = ""
        #print(bestseller_book["year"])
        try:
            bestseller_book["Price"] = book.find("span", class_="sale-price").get_text().split("€")[0].replace(",", ".").replace(" ", "")
        except AttributeError:
                  bestseller_book["Price"] = ""
    
        #print(bestseller_book)
        #print(count)
        bestseller_books.append(bestseller_book.copy())
        page = page + 1
        
print(bestseller_books)

[{'Title': 'The Little Book of Common Sense Investing', 'Format': 'Hardback', 'Year': '2017', 'Price': '16.80'}, {'Title': "It Ends With Us: The most heartbreaking novel you'll ever read", 'Format': 'Paperback', 'Year': '2016', 'Price': '7.37'}, {'Title': 'Before the Coffee Gets Cold', 'Format': 'Paperback', 'Year': '2019', 'Price': '9.47'}, {'Title': 'Verity', 'Format': 'Paperback', 'Year': '2022', 'Price': '6.58'}, {'Title': 'Undaunted: Stalingrad', 'Format': 'Game', 'Year': '2022', 'Price': ''}, {'Title': 'Me vs Brain', 'Format': 'Hardback', 'Year': '2023', 'Price': '21.48'}, {'Title': 'Atomic Habits', 'Format': 'Paperback', 'Year': '2018', 'Price': '15.27'}, {'Title': 'Four Thousand Weeks', 'Format': 'Paperback', 'Year': '2022', 'Price': '9.86'}, {'Title': 'The Tools', 'Format': 'Paperback', 'Year': '2014', 'Price': '17.18'}, {'Title': 'Seven Husbands of Evelyn Hugo', 'Format': 'Paperback', 'Year': '2021', 'Price': '7.43'}, {'Title': 'The Midnight Library', 'Format': 'Paperback', '

In [56]:
# Dictionary with count
count = 0
for book in bestseller_books:
    count +=1
    print(book)
    print(count)

    

{'Title': 'The Little Book of Common Sense Investing', 'Format': 'Hardback', 'Year': '2017', 'Price': '16.80'}
1
{'Title': "It Ends With Us: The most heartbreaking novel you'll ever read", 'Format': 'Paperback', 'Year': '2016', 'Price': '7.37'}
2
{'Title': 'Before the Coffee Gets Cold', 'Format': 'Paperback', 'Year': '2019', 'Price': '9.47'}
3
{'Title': 'Verity', 'Format': 'Paperback', 'Year': '2022', 'Price': '6.58'}
4
{'Title': 'Undaunted: Stalingrad', 'Format': 'Game', 'Year': '2022', 'Price': ''}
5
{'Title': 'Me vs Brain', 'Format': 'Hardback', 'Year': '2023', 'Price': '21.48'}
6
{'Title': 'Atomic Habits', 'Format': 'Paperback', 'Year': '2018', 'Price': '15.27'}
7
{'Title': 'Four Thousand Weeks', 'Format': 'Paperback', 'Year': '2022', 'Price': '9.86'}
8
{'Title': 'The Tools', 'Format': 'Paperback', 'Year': '2014', 'Price': '17.18'}
9
{'Title': 'Seven Husbands of Evelyn Hugo', 'Format': 'Paperback', 'Year': '2021', 'Price': '7.43'}
10
{'Title': 'The Midnight Library', 'Format': 'Pap

In [57]:
# Converted into DataFrame
books_df = pd.DataFrame(bestseller_books)
books_df

,Title,Format,Year,Price
0,The Little Book of Common Sense Investing,Hardback,2017,16.80
1,It Ends With Us: The most heartbreaking novel ...,Paperback,2016,7.37
2,Before the Coffee Gets Cold,Paperback,2019,9.47
3,Verity,Paperback,2022,6.58
4,Undaunted: Stalingrad,Game,2022,
...,...,...,...,...
1045,Tribe of Mentors,Paperback,2017,17.41
1046,The Art & Science of Foodpairing,Hardback,2020,
1047,Bad Dreams and Other Stories,Paperback,2018,11.72
1048,Dreams for our Daughters,Paperback,2022,7.13


In [210]:
#Question1: How many books sold out in year 2022?
books_df[books_df["Year"] == "2022" ]

,Title,Format,Year,Price
679,The Kiss of Deception,Paperback,2022,9.90
665,Blue Lock 2,Paperback,2022,12.33
668,MEZCLA,Hardback,2022,23.85
247,We Are the ARK: Returning Our Gardens to Their...,Hardback,2022,22.99
677,Good Boundaries and Goodbyes,Paperback,2022,14.25
...,...,...,...,...
166,the Silva Mind Control Method,Paperback,2022,17.02
170,Fairy Tale,Hardback,2022,15.87
139,Diddly Squat,Paperback,2022,8.12
165,If We Were Villains - Illustrated edition,Hardback,2022,18.11


In [224]:
#Question2: Which are the top 10 expensive books?

top10 = books_df.sort_values('Price',ascending=False,inplace=False , axis=0,ignore_index=True)
(top10.head(10))


,Title,Format,Year,Price
0,The Lincoln Highway,Paperback,2022,9.99
1,The Heart's Invisible Furies,Paperback,2017,9.97
2,Rich Dad Poor Dad,Paperback,2018,9.97
3,Cloud Cuckoo Land,Paperback,2022,9.97
4,Meditations,Paperback,2019,9.96
5,The Snowman,Paperback,1987,9.96
6,The Christmas Bookshop,Paperback,2022,9.96
7,The Day I Got Trapped In My Brain,Paperback,2022,9.94
8,The Courage To Be Disliked,Paperback,2019,9.93
9,Bloodmarked,Paperback,2022,9.92


In [60]:
#Question3 : How many books are sold with thde format type 'Paperback'?
books_df[books_df["Format"] == "Paperback" ]

,Title,Format,Year,Price
1,It Ends With Us: The most heartbreaking novel ...,Paperback,2016,7.37
2,Before the Coffee Gets Cold,Paperback,2019,9.47
3,Verity,Paperback,2022,6.58
6,Atomic Habits,Paperback,2018,15.27
7,Four Thousand Weeks,Paperback,2022,9.86
...,...,...,...,...
1043,Call Me By Your Name,Paperback,2017,5.97
1045,Tribe of Mentors,Paperback,2017,17.41
1047,Bad Dreams and Other Stories,Paperback,2018,11.72
1048,Dreams for our Daughters,Paperback,2022,7.13


In [225]:
#Question4: Book who has title as 'Undaunted: Stalingrad' has sold out till date?
books_df[books_df["Title"]== "Undaunted: Stalingrad" ]

,Title,Format,Year,Price
4,Undaunted: Stalingrad,Game,2022,
34,Undaunted: Stalingrad,Game,2022,


In [174]:
books_df.to_csv('BestSeller_book.csv')

In [176]:
data = pd.read_csv('BestSeller_book.csv', float_precision='round_trip')
data

,Unnamed: 0,Title,Format,Year,Price
0,0,The Little Book of Common Sense Investing,Hardback,2017.0,16.80
1,1,It Ends With Us: The most heartbreaking novel ...,Paperback,2016.0,7.37
2,2,Before the Coffee Gets Cold,Paperback,2019.0,9.47
3,3,Verity,Paperback,2022.0,6.58
4,4,Undaunted: Stalingrad,Game,2022.0,NaN
...,...,...,...,...,...
1045,1045,Tribe of Mentors,Paperback,2017.0,17.41
1046,1046,The Art & Science of Foodpairing,Hardback,2020.0,NaN
1047,1047,Bad Dreams and Other Stories,Paperback,2018.0,11.72
1048,1048,Dreams for our Daughters,Paperback,2022.0,7.13


In [177]:
#Output the total number of rows and columns.
rows = data.shape[0]
cols = data.shape[1]
print("Rows: " + str(rows))
print("Columns: " + str(cols))

Rows: 1050
Columns: 5


In [178]:
# Output the number of non-null rows (by column)
data.notnull().sum()

Unnamed: 0    1050
Title         1050
Format        1050
Year          1049
Price         1039
dtype: int64

In [179]:
# Output the number of null values (by column).
data.isnull().sum()

Unnamed: 0     0
Title          0
Format         0
Year           1
Price         11
dtype: int64

In [180]:
#Output the number of null values for all columns.
data.isnull().sum().sum()

12

In [182]:
average_mean_1 = data["Year"].astype("float").mean(axis=0)
print("Average", "%.2f" %average_mean_1)

Average 2018.13


In [183]:
data['Year'].replace(np.NaN, average_mean_1,inplace=True)
data.head()

,Unnamed: 0,Title,Format,Year,Price
0,0,The Little Book of Common Sense Investing,Hardback,2017.0,16.80
1,1,It Ends With Us: The most heartbreaking novel ...,Paperback,2016.0,7.37
2,2,Before the Coffee Gets Cold,Paperback,2019.0,9.47
3,3,Verity,Paperback,2022.0,6.58
4,4,Undaunted: Stalingrad,Game,2022.0,NaN


In [184]:
average_mean_1 = data["Price"].astype("float").mean(axis=0)
print("Average", "%.2f" %average_mean_1)

Average 13.92


In [185]:
data['Price'].replace(np.NaN, price_avg,inplace=True)
data.head()

,Unnamed: 0,Title,Format,Year,Price
0,0,The Little Book of Common Sense Investing,Hardback,2017.0,16.8
1,1,It Ends With Us: The most heartbreaking novel ...,Paperback,2016.0,7.37
2,2,Before the Coffee Gets Cold,Paperback,2019.0,9.47
3,3,Verity,Paperback,2022.0,6.58
4,4,Undaunted: Stalingrad,Game,2022.0,13.92


In [186]:
data

,Unnamed: 0,Title,Format,Year,Price
0,0,The Little Book of Common Sense Investing,Hardback,2017.0,16.8
1,1,It Ends With Us: The most heartbreaking novel ...,Paperback,2016.0,7.37
2,2,Before the Coffee Gets Cold,Paperback,2019.0,9.47
3,3,Verity,Paperback,2022.0,6.58
4,4,Undaunted: Stalingrad,Game,2022.0,13.92
...,...,...,...,...,...
1045,1045,Tribe of Mentors,Paperback,2017.0,17.41
1046,1046,The Art & Science of Foodpairing,Hardback,2020.0,13.92
1047,1047,Bad Dreams and Other Stories,Paperback,2018.0,11.72
1048,1048,Dreams for our Daughters,Paperback,2022.0,7.13


In [187]:
data.dtypes

Unnamed: 0      int64
Title          object
Format         object
Year          float64
Price          object
dtype: object

In [188]:
data.isnull().sum()

Unnamed: 0    0
Title         0
Format        0
Year          0
Price         0
dtype: int64

In [190]:
data.Year = data.Year.astype(int)

In [191]:
data

,Unnamed: 0,Title,Format,Year,Price
0,0,The Little Book of Common Sense Investing,Hardback,2017,16.8
1,1,It Ends With Us: The most heartbreaking novel ...,Paperback,2016,7.37
2,2,Before the Coffee Gets Cold,Paperback,2019,9.47
3,3,Verity,Paperback,2022,6.58
4,4,Undaunted: Stalingrad,Game,2022,13.92
...,...,...,...,...,...
1045,1045,Tribe of Mentors,Paperback,2017,17.41
1046,1046,The Art & Science of Foodpairing,Hardback,2020,13.92
1047,1047,Bad Dreams and Other Stories,Paperback,2018,11.72
1048,1048,Dreams for our Daughters,Paperback,2022,7.13
